In [110]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

import sqlite3
import csv

In [104]:
#VARIABLES

#Postgres credentials
pw = 'npF5SlOzvW'
db = "d2b_assessment" 
user='byroneji4734'
host='d2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com'
port = '5432'


#s3 credentials
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket_name =  "d2b-internal-assessment-bucket"
Prefix='orders_data/'

res = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix=Prefix)
    

#Storage Locations
source_path = "../data/Webpages/"
save_path = "output/"
index_file = '../'    



In [103]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:

        connection = psycopg2.connect(
            database=db,
            user=user,
            password=pw,
            host=host,
            port=port
        )
        
        print("PostgreSql Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [109]:
#List and Download files from data lake



def download_csv_from_s3_bucket():
    file_list = []
    for content in res.get('Contents', [])[1:]:
        trunc_key = (content['Key'].rsplit(Prefix))
        trunc_key = (','.join(trunc_key[1:]))
        #print(trunc_key)
        s3.download_file(bucket_name,content['Key'], trunc_key)
        file_list.append(trunc_key[:-4]) #create list of files ignoring the file extension(.csv)

    for file in file_list:
        s3.download_file(bucket_name, "orders_data/"+file+".csv", file+".csv")
    return file_list

['orders', 'reviews', 'shipment_deliveries']

In [131]:
def read_and_convert_csv_to_dataframe(df_name,csv_path):

    
    if df_name == 'shipment_deliveries':
        df_name = pd.read_csv(csv_path)
        df_name = pd.DataFrame(df_name)
        #replace NaN values with a weird date string to enable SQL accept the entry
        df_name = df_name.replace(np.nan, '1000-01-01',regex = True) 
   
    else:
        df_name = pd.read_csv(csv_path)
        df_name = pd.DataFrame(df_name)
    return df_name               


### LOAD DATA TO STAGING SCHEMA

In [118]:
def load_df_to_postgres(df, table):

    tuples = [list(row) for row in df.itertuples(index=False)]
 

    cols = ','.join(list(df.columns))
    
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)                   
    cursor = connection.cursor()
    
    try:
        extras.execute_values(cursor, query, tuples)
        #cursor.execute(query)
        connection.commit()
      
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return 1
    except psycopg2.InterfaceError:
        pass
    print("load to postgresql successful")
    cursor.close()

# SQL TRANSFORMATION

In [ ]:
%%read_sql
--Replace strange date values to Null in the date columns

UPDATE byroneji4734_staging.shipment_deliveries
    SET shipment_date = Null
WHERE shipment_date = '1000-01-01';


UPDATE byroneji4734_staging.shipment_deliveries
    SET delivery_date = Null
WHERE delivery_date = '1000-01-01';


***1) The total number of orders placed on a public holiday every month, for the past year***

In [98]:
%%read_sql

SELECT
    CURRENT_DATE AS ingestion_date, 
    count(order_id) FILTER (WHERE month_of_the_year_num = 1) AS "tt_order_hol_jan",
    count(order_id) FILTER (WHERE month_of_the_year_num = 2) AS "tt_order_hol_feb",
    count(order_id) FILTER (WHERE month_of_the_year_num = 3) AS "tt_order_hol_mar",
    count(order_id) FILTER (WHERE month_of_the_year_num = 4) AS "tt_order_hol_apr",
    count(order_id) FILTER (WHERE month_of_the_year_num = 5) AS "tt_order_hol_may",
    count(order_id) FILTER (WHERE month_of_the_year_num = 6) AS "tt_order_hol_jun",
    count(order_id) FILTER (WHERE month_of_the_year_num = 7) AS "tt_order_hol_jul",
    count(order_id) FILTER (WHERE month_of_the_year_num = 8) AS "tt_order_hol_aug",
    count(order_id) FILTER (WHERE month_of_the_year_num = 9) AS "tt_order_hol_sep",
    count(order_id) FILTER (WHERE month_of_the_year_num = 10) AS "tt_order_hol_oct",
    count(order_id) FILTER (WHERE month_of_the_year_num = 11) AS "tt_order_hol_nov",
    count(order_id) FILTER (WHERE month_of_the_year_num = 12) AS "tt_order_hol_dec"
FROM if_common.dim_dates
JOIN byroneji4734_staging.orders
ON order_date = calendar_dt
WHERE orders.order_date IN(

    SELECT calendar_dt
    FROM if_common.dim_dates
    WHERE calendar_dt >= CURRENT_DATE - INTERVAL '1 year'
    AND calendar_dt <= CURRENT_DATE
    AND working_day = False
    AND day_of_the_week_num BETWEEN 1 AND 5
    GROUP BY month_of_the_year_num, calendar_dt
    ORDER BY month_of_the_year_num
)


Query started at 12:29:00 PM W. Central Africa Standard Time

InterfaceError: connection already closed

***2) Total number of Late & undelivered shipments***

In [12]:
%%read_sql

SELECT CURRENT_DATE AS ingestion_date, tt_late_shipments,  tt_undelivered_items
FROM 
(
    SELECT count(orders.order_date) AS tt_late_shipments
    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date >= (orders.order_date + 6)
    AND shipment_deliveries.delivery_date IS Null
) 
    as x, 

(

    SELECT count(orders.order_date) as tt_undelivered_items
    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date IS Null
    AND shipment_deliveries.delivery_date IS Null
    AND (order_date) + 15 = '2022-09-05'

) 

as y

Query started at 10:11:38 PM W. Central Africa Standard Time; Query executed in 0.00 m

,ingestion_date,tt_late_shipments,tt_undelivered_items
0,2022-09-27,175,11


In [128]:
%%read_sql

SELECT *

     

FROM

(

SELECT CURRENT_DATE AS ingestion_date, product_name, calendar_dt AS most_ordered_day,
     bool_and(working_day = False AND day_of_the_week_num BETWEEN 1 AND 5)AS is_public_holiday,
     sum(review) AS tt_review_points


FROM if_common.dim_dates

FULL JOIN byroneji4734_staging.shipment_deliveries
ON shipment_deliveries.delivery_date = calendar_dt


JOIN byroneji4734_staging.reviews
ON reviews.product_id = 
(
    SELECT reviews.product_id
    FROM byroneji4734_staging.reviews
    GROUP BY reviews.product_id
    ORDER BY sum(review) DESC
    LIMIT 1


)

LEFT JOIN if_common.dim_products
ON dim_products.product_id = reviews.product_id

WHERE dim_dates.calendar_dt = 
(

SELECT order_date
FROM byroneji4734_staging.orders


WHERE orders.product_id = 
(
    SELECT reviews.product_id
    FROM byroneji4734_staging.reviews
    GROUP BY reviews.product_id
    ORDER BY sum(review) DESC
    LIMIT 1
    
)

GROUP BY  order_date, orders.product_id
ORDER BY sum (quantity) DESC
LIMIT 1

)
GROUP BY calendar_dt, product_name, reviews.product_id

) AS a,



(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_one_star_review

FROM byroneji4734_staging.reviews
WHERE review = 1
GROUP BY  product_id, review
LIMIT 1
     
) AS b,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review=1), 3) as pct_two_star_review

FROM byroneji4734_staging.reviews
WHERE review = 2
GROUP BY  product_id, review
LIMIT 1
     
) AS c,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_three_star_review

FROM byroneji4734_staging.reviews
WHERE review = 3
GROUP BY  product_id, review
LIMIT 1
     
) AS d,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_four_star_review

FROM byroneji4734_staging.reviews
WHERE review = 4
GROUP BY  product_id, review
LIMIT 1
     
) AS e,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_five_star_review

FROM byroneji4734_staging.reviews
WHERE review = 5
GROUP BY  product_id, review
LIMIT 1
     
) AS f,


(


     SELECT    1 - (count(orders.order_id) FILTER (WHERE shipment_date >= (orders.order_date + 6)
                AND shipment_deliveries.delivery_date IS Null AND orders.product_id = (

                            SELECT reviews.product_id
                            FROM byroneji4734_staging.reviews
                            GROUP BY reviews.product_id
                            ORDER BY sum(review) DESC
                            LIMIT 1

                )
                
                )/CAST(COUNT(orders.order_id) AS float)) AS pct_early_shipments,


                ((count(orders.order_id) FILTER (WHERE shipment_date >= (orders.order_date + 6)
                AND shipment_deliveries.delivery_date IS Null AND orders.product_id = (

                            SELECT reviews.product_id
                            FROM byroneji4734_staging.reviews
                            GROUP BY reviews.product_id
                            ORDER BY sum(review) DESC
                            LIMIT 1

                )
                
                )/CAST(COUNT(orders.order_id) AS float))) AS pct_late_shipments        
    


    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date >= (orders.order_date + 6)
    AND shipment_deliveries.delivery_date IS Null





) as g


Query started at 03:26:35 AM W. Central Africa Standard Time; Query executed in 0.01 m

,ingestion_date,product_name,most_ordered_day,is_public_holiday,tt_review_points,pct_one_star_review,pct_two_star_review,pct_three_star_review,pct_four_star_review,pct_five_star_review,pct_early_shipments,pct_late_shipments
0,2022-09-28,Scooter,2022-01-06,False,1934,0.037,0.045,0.035,0.04,0.046,0.965714,0.034286


In [97]:
def read_and_convert_query_to_dataframe(query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        
        field_names = [i[0] for i in cursor.description ]
        result = pd.DataFrame(result,columns=field_names)
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [142]:
def update_dataframe(query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('weird dates casted to Null')
    except Error as err:
        print(f"Error: '{err}'")

In [129]:
query_update_for_null_shipment_date = """
UPDATE byroneji4734_staging.shipment_deliveries
    SET shipment_date = Null
WHERE shipment_date = '1000-01-01';
"""

In [130]:
query_update_for_null_delivery_date = """
UPDATE byroneji4734_staging.shipment_deliveries
    SET delivery_date = Null
WHERE delivery_date = '1000-01-01';
"""

In [66]:
query_agg_shipments = """
SELECT CURRENT_DATE AS ingestion_date, tt_late_shipments,  tt_undelivered_items
FROM 
(
    SELECT count(orders.order_date) AS tt_late_shipments
    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date >= (orders.order_date + 6)
    AND shipment_deliveries.delivery_date IS Null
) 
    as x, 

(

    SELECT count(orders.order_date) as tt_undelivered_items
    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date IS Null
    AND shipment_deliveries.delivery_date IS Null
    AND (order_date) + 15 = '2022-09-05'

) 

as y
"""

In [86]:
query_agg_public_holiday = """
SELECT
    CURRENT_DATE AS ingestion_date, 
    count(order_id) FILTER (WHERE month_of_the_year_num = 1) AS "tt_order_hol_jan",
    count(order_id) FILTER (WHERE month_of_the_year_num = 2) AS "tt_order_hol_feb",
    count(order_id) FILTER (WHERE month_of_the_year_num = 3) AS "tt_order_hol_mar",
    count(order_id) FILTER (WHERE month_of_the_year_num = 4) AS "tt_order_hol_apr",
    count(order_id) FILTER (WHERE month_of_the_year_num = 5) AS "tt_order_hol_may",
    count(order_id) FILTER (WHERE month_of_the_year_num = 6) AS "tt_order_hol_jun",
    count(order_id) FILTER (WHERE month_of_the_year_num = 7) AS "tt_order_hol_jul",
    count(order_id) FILTER (WHERE month_of_the_year_num = 8) AS "tt_order_hol_aug",
    count(order_id) FILTER (WHERE month_of_the_year_num = 9) AS "tt_order_hol_sep",
    count(order_id) FILTER (WHERE month_of_the_year_num = 10) AS "tt_order_hol_oct",
    count(order_id) FILTER (WHERE month_of_the_year_num = 11) AS "tt_order_hol_nov",
    count(order_id) FILTER (WHERE month_of_the_year_num = 12) AS "tt_order_hol_dec"
FROM if_common.dim_dates
JOIN byroneji4734_staging.orders
ON order_date = dim_dates.calendar_dt
WHERE orders.order_date IN(

    SELECT dim_dates.calendar_dt
    FROM if_common.dim_dates
    WHERE dim_dates.calendar_dt >= CURRENT_DATE - INTERVAL '1 year'
    AND dim_dates.calendar_dt <= CURRENT_DATE
    AND working_day = False
    AND day_of_the_week_num BETWEEN 1 AND 5
    GROUP BY month_of_the_year_num, dim_dates.calendar_dt
    ORDER BY month_of_the_year_num
)

"""

In [91]:
query_best_performing_product = """
SELECT *

     

FROM

(

SELECT CURRENT_DATE AS ingestion_date, product_name, calendar_dt AS most_ordered_day,
     bool_and(working_day = False AND day_of_the_week_num BETWEEN 1 AND 5)AS is_public_holiday,
     sum(review) AS tt_review_points


FROM if_common.dim_dates

FULL JOIN byroneji4734_staging.shipment_deliveries
ON shipment_deliveries.delivery_date = calendar_dt


JOIN byroneji4734_staging.reviews
ON reviews.product_id = 
(
    SELECT reviews.product_id
    FROM byroneji4734_staging.reviews
    GROUP BY reviews.product_id
    ORDER BY sum(review) DESC
    LIMIT 1


)

LEFT JOIN if_common.dim_products
ON dim_products.product_id = reviews.product_id

WHERE dim_dates.calendar_dt = 
(

SELECT order_date
FROM byroneji4734_staging.orders


WHERE orders.product_id = 
(
    SELECT reviews.product_id
    FROM byroneji4734_staging.reviews
    GROUP BY reviews.product_id
    ORDER BY sum(review) DESC
    LIMIT 1
    
)

GROUP BY  order_date, orders.product_id
ORDER BY sum (quantity) DESC
LIMIT 1

)
GROUP BY calendar_dt, product_name, reviews.product_id

) AS a,



(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_one_star_review

FROM byroneji4734_staging.reviews
WHERE review = 1
GROUP BY  product_id, review
LIMIT 1
     
) AS b,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review=1), 3) as pct_two_star_review

FROM byroneji4734_staging.reviews
WHERE review = 2
GROUP BY  product_id, review
LIMIT 1
     
) AS c,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_three_star_review

FROM byroneji4734_staging.reviews
WHERE review = 3
GROUP BY  product_id, review
LIMIT 1
     
) AS d,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_four_star_review

FROM byroneji4734_staging.reviews
WHERE review = 4
GROUP BY  product_id, review
LIMIT 1
     
) AS e,


(

SELECT 
                  ROUND(COUNT(*) * 1.0 / SUM(COUNT(*)) 
          OVER (PARTITION BY review), 3) as pct_five_star_review

FROM byroneji4734_staging.reviews
WHERE review = 5
GROUP BY  product_id, review
LIMIT 1
     
) AS f,


(


     SELECT    1 - (count(orders.order_id) FILTER (WHERE shipment_date >= (orders.order_date + 6)
                AND shipment_deliveries.delivery_date IS Null AND orders.product_id = (

                            SELECT reviews.product_id
                            FROM byroneji4734_staging.reviews
                            GROUP BY reviews.product_id
                            ORDER BY sum(review) DESC
                            LIMIT 1

                )
                
                )/CAST(COUNT(orders.order_id) AS float)) AS pct_early_shipments,


                ((count(orders.order_id) FILTER (WHERE shipment_date >= (orders.order_date + 6)
                AND shipment_deliveries.delivery_date IS Null AND orders.product_id = (

                            SELECT reviews.product_id
                            FROM byroneji4734_staging.reviews
                            GROUP BY reviews.product_id
                            ORDER BY sum(review) DESC
                            LIMIT 1

                )
                
                )/CAST(COUNT(orders.order_id) AS float))) AS pct_late_shipments        
    


    FROM byroneji4734_staging.shipment_deliveries

    JOIN byroneji4734_staging.orders
    ON orders.order_id = shipment_deliveries.order_id
    WHERE shipment_date >= (orders.order_date + 6)
    AND shipment_deliveries.delivery_date IS Null





) as g

"""

In [96]:
#READ QUERIES INTO PYTHON FUNCTION AND CONVERT TO DATAFRAME

agg_shipments = read_and_convert_query_to_dataframe(query_agg_shipments)
agg_public_holiday = read_and_convert_query_to_dataframe(query_agg_public_holiday)
best_performing_product = read_and_convert_query_to_dataframe(query_best_performing_product)

#agg_public_holiday
#best_performing_product
#agg_shipments

#columns

,ingestion_date,tt_late_shipments,tt_undelivered_items
0,2022-09-28,175,11


In [ ]:
if __name__ == "__main__":

    connection = create_server_connection(host, user, pw)

    download_csv_from_s3_bucket()

    orders = read_and_convert_csv_to_dataframe('orders','orders.csv')

    reviews = read_and_convert_csv_to_dataframe('reviews','reviews.csv')

    shipment_deliveries = read_and_convert_csv_to_dataframe('shipment_deliveries','shipment_deliveries.csv')

    load_df_to_postgres(orders, 'byroneji4734_staging.orders')

    load_df_to_postgres(reviews, 'byroneji4734_staging.reviews')

    load_df_to_postgres(shipment_deliveries, 'byroneji4734_staging.shipment_deliveries')

    update_dataframe(query_update_for_null_shipment_date)

    update_dataframe(query_update_for_null_delivery_date)